In [15]:
import numpy as np 
import csv
import matplotlib.pyplot as plt
import os
from copy import deepcopy
np.random.seed(40)
path='../数据'
#files=os.listdir(path)
files=['ib1000_2.csv','ib600_2.csv','ib800_2.csv','n1000_3_2.csv','n600_3_2.csv',
       'n800_3_2.csv','ob1000_2.csv', 'ob600_2.csv','ob800_2.csv', 'tb1000_2.csv', 
       'tb600_2.csv','tb800_2.csv']
temp_data=[]
temp_data_all=[]
train_input=[]
test_input=[]
for item in files:
   if 'csv' in item:
       if '800' not in item:
           print ('train:',item)
           root=os.path.join(path,item)
           csv_reader = csv.reader(open(root, encoding='utf-8'))
           for row in csv_reader:
               temp_data.append(row[0])
           temp_data=np.asarray(temp_data)
           temp_data=temp_data.astype(np.float64)
           if temp_data.shape[0]==500500:        
               train_input.append(temp_data[:500000])
           else:
               train_input.append(temp_data[:1500000])
           temp_data=[]
       if '800' in item:
           print ('test:',item)
           root=os.path.join(path,item)
           csv_reader = csv.reader(open(root, encoding='utf-8'))
           for row in csv_reader:
               temp_data.append(row[0])
           temp_data=np.asarray(temp_data)
           temp_data=temp_data.astype(np.float64)
           if temp_data.shape[0]==500500:        
               test_input.append(temp_data[:500000])
           else:
               test_input.append(temp_data[:1500000])
           temp_data=[]      
         
train_data=[]
test_data=[]
for item in train_input:
    item=item.reshape(-1,10000)
    train_data.append(item)
for item in test_input:
    item=item.reshape(-1,10000)  
    test_data.append(item)

a=deepcopy(train_data[0])
for index in range(1,len(train_data)):
    a=np.concatenate((a,train_data[index]))
train_data=deepcopy(a)

a=deepcopy(test_data[0])
for index in range(1,len(test_data)):
    a=np.concatenate((a,test_data[index]))
test_data=deepcopy(a)

label0=np.zeros((100,1))
label1=np.ones((300,1))
label2=np.zeros((100,1))
label2[:,:]=2
label3=np.zeros((100,1))
label3[:,:]=3
train_label=np.concatenate((label0,label1,label2,label3))

label0=np.zeros((50,1))
label1=np.ones((150,1))
label2=np.zeros((50,1))
label2[:,:]=2
label3=np.zeros((50,1))
label3[:,:]=3
test_label=np.concatenate((label0,label1,label2,label3))

train: ib1000_2.csv
train: ib600_2.csv
test: ib800_2.csv
train: n1000_3_2.csv
train: n600_3_2.csv
test: n800_3_2.csv
train: ob1000_2.csv
train: ob600_2.csv
test: ob800_2.csv
train: tb1000_2.csv
train: tb600_2.csv
test: tb800_2.csv


In [6]:
train_data.shape
test_data.shape
test_label.shape
train_label.shape

(600, 1)

In [16]:
from keras.utils import to_categorical
train_label=to_categorical(train_label)
test_label=to_categorical(test_label)

from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras import optimizers
#from sklearn.cross_validation import train_test_split


#mean_value=np.mean(x_train,axis=0)
#max_value=np.max(x_train,axis=0)
#min_value=np.min(x_train,axis=0)
#x_train=(x_train-mean_value)/(max_value-min_value)
#x_test=(x_test-mean_value)/(max_value-min_value)
#from sklearn import preprocessing
#scaler= preprocessing.StandardScaler().fit(x_train)
#train_data=scaler.transform(x_train)
#test_data=scaler.transform(x_test)

train_data=np.expand_dims(train_data,axis=2)
test_data=np.expand_dims(test_data,axis=2)

input=Input(shape=(10000,1))
conv1=Conv1D(32,kernel_size=1000,strides=10,activation='relu')(input)
pool1=MaxPool1D(pool_size=2)(conv1)
conv2=Conv1D(64,kernel_size=2,strides=1,activation='relu')(pool1)
pool2=MaxPool1D(pool_size=2)(conv2)
#conv3=Conv1D(64,kernel_size=2,activation='relu')(pool2)
#pool3=MaxPool1D(pool_size=2)(conv3)



flat1=Flatten(name='flatten')(pool2)

dense2=Dense(50,activation='relu')(flat1)
dense3=Dropout(0.6)(dense2)
#dense4=Dense(50,activation='relu')(dense2)

output=Dense(4,activation='softmax',name='result')(dense3)

model=Model(inputs=input,outputs=output)


optimizer = optimizers.adam(lr=0.001)
model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
model.summary()
history=model.fit(train_data,train_label, epochs=500,validation_data=(test_data,test_label))
dense1_layer_model = Model(inputs=model.input,  
                           outputs=model.get_layer('result').output)
predict = dense1_layer_model.predict(train_data)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 10000, 1)          0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 901, 32)           32032     
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 450, 32)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 449, 64)           4160      
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 224, 64)           0         
_________________________________________________________________
flatten (Flatten)            (None, 14336)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                716850    
__________

Epoch 50/500
600/600 [==============================] - 0s 148us/step - loss: 0.1023 - acc: 0.9383 - val_loss: 0.6645 - val_acc: 0.8933
Epoch 51/500
600/600 [==============================] - 0s 154us/step - loss: 0.0973 - acc: 0.9467 - val_loss: 0.4380 - val_acc: 0.9200
Epoch 52/500
600/600 [==============================] - 0s 153us/step - loss: 0.0838 - acc: 0.9533 - val_loss: 0.6113 - val_acc: 0.9000
Epoch 53/500
600/600 [==============================] - 0s 153us/step - loss: 0.0826 - acc: 0.9517 - val_loss: 0.7924 - val_acc: 0.8833
Epoch 54/500
600/600 [==============================] - 0s 149us/step - loss: 0.0752 - acc: 0.9550 - val_loss: 0.5889 - val_acc: 0.9133
Epoch 55/500
600/600 [==============================] - 0s 155us/step - loss: 0.0752 - acc: 0.9550 - val_loss: 1.0201 - val_acc: 0.8833
Epoch 56/500
600/600 [==============================] - 0s 151us/step - loss: 0.0871 - acc: 0.9483 - val_loss: 0.3226 - val_acc: 0.9367
Epoch 57/500
600/600 [==========================

600/600 [==============================] - 0s 154us/step - loss: 0.0511 - acc: 0.9783 - val_loss: 0.6187 - val_acc: 0.9300
Epoch 111/500
600/600 [==============================] - 0s 148us/step - loss: 0.0515 - acc: 0.9783 - val_loss: 0.3866 - val_acc: 0.9467
Epoch 112/500
600/600 [==============================] - 0s 152us/step - loss: 0.0490 - acc: 0.9767 - val_loss: 0.7190 - val_acc: 0.9267
Epoch 113/500
600/600 [==============================] - 0s 143us/step - loss: 0.0681 - acc: 0.9767 - val_loss: 0.8979 - val_acc: 0.9067
Epoch 114/500
600/600 [==============================] - 0s 151us/step - loss: 0.0547 - acc: 0.9800 - val_loss: 0.6111 - val_acc: 0.9300
Epoch 115/500
600/600 [==============================] - 0s 151us/step - loss: 0.0558 - acc: 0.9783 - val_loss: 0.5236 - val_acc: 0.9367
Epoch 116/500
600/600 [==============================] - 0s 151us/step - loss: 0.0562 - acc: 0.9733 - val_loss: 0.4663 - val_acc: 0.9433
Epoch 117/500
600/600 [==============================] 

Epoch 170/500
600/600 [==============================] - 0s 144us/step - loss: 0.0537 - acc: 0.9733 - val_loss: 0.4239 - val_acc: 0.9367
Epoch 171/500
600/600 [==============================] - 0s 143us/step - loss: 0.0838 - acc: 0.9683 - val_loss: 0.4584 - val_acc: 0.9300
Epoch 172/500
600/600 [==============================] - 0s 142us/step - loss: 0.0528 - acc: 0.9817 - val_loss: 0.4759 - val_acc: 0.9367
Epoch 173/500
600/600 [==============================] - 0s 146us/step - loss: 0.0539 - acc: 0.9883 - val_loss: 0.6464 - val_acc: 0.9200
Epoch 174/500
600/600 [==============================] - 0s 143us/step - loss: 0.0560 - acc: 0.9783 - val_loss: 0.8998 - val_acc: 0.9200
Epoch 175/500
600/600 [==============================] - 0s 143us/step - loss: 0.0576 - acc: 0.9750 - val_loss: 0.8678 - val_acc: 0.9267
Epoch 176/500
600/600 [==============================] - 0s 147us/step - loss: 0.0543 - acc: 0.9817 - val_loss: 0.6868 - val_acc: 0.9433
Epoch 177/500
600/600 [==================

Epoch 230/500
600/600 [==============================] - 0s 146us/step - loss: 0.0547 - acc: 0.9833 - val_loss: 0.2132 - val_acc: 0.9733
Epoch 231/500
600/600 [==============================] - 0s 143us/step - loss: 0.0595 - acc: 0.9817 - val_loss: 0.2539 - val_acc: 0.9600
Epoch 232/500
600/600 [==============================] - 0s 143us/step - loss: 0.0739 - acc: 0.9733 - val_loss: 0.2465 - val_acc: 0.9667
Epoch 233/500
600/600 [==============================] - 0s 142us/step - loss: 0.0584 - acc: 0.9767 - val_loss: 0.4937 - val_acc: 0.9467
Epoch 234/500
600/600 [==============================] - 0s 145us/step - loss: 0.0410 - acc: 0.9833 - val_loss: 0.3169 - val_acc: 0.9567
Epoch 235/500
600/600 [==============================] - 0s 153us/step - loss: 0.0420 - acc: 0.9900 - val_loss: 0.4526 - val_acc: 0.9433
Epoch 236/500
600/600 [==============================] - 0s 151us/step - loss: 0.0518 - acc: 0.9800 - val_loss: 0.4180 - val_acc: 0.9533
Epoch 237/500
600/600 [==================

Epoch 290/500
600/600 [==============================] - 0s 150us/step - loss: 0.0833 - acc: 0.9633 - val_loss: 0.3694 - val_acc: 0.9533
Epoch 291/500
600/600 [==============================] - 0s 152us/step - loss: 0.0555 - acc: 0.9767 - val_loss: 0.1886 - val_acc: 0.9833
Epoch 292/500
600/600 [==============================] - 0s 149us/step - loss: 0.0515 - acc: 0.9783 - val_loss: 0.3794 - val_acc: 0.9600
Epoch 293/500
600/600 [==============================] - 0s 154us/step - loss: 0.0441 - acc: 0.9817 - val_loss: 0.4796 - val_acc: 0.9533
Epoch 294/500
600/600 [==============================] - 0s 151us/step - loss: 0.0541 - acc: 0.9800 - val_loss: 0.6434 - val_acc: 0.9333
Epoch 295/500
600/600 [==============================] - 0s 153us/step - loss: 0.0534 - acc: 0.9817 - val_loss: 0.6609 - val_acc: 0.9400
Epoch 296/500
600/600 [==============================] - 0s 148us/step - loss: 0.0288 - acc: 0.9900 - val_loss: 0.5482 - val_acc: 0.9433
Epoch 297/500
600/600 [==================

Epoch 350/500
600/600 [==============================] - 0s 146us/step - loss: 0.0527 - acc: 0.9833 - val_loss: 0.4522 - val_acc: 0.9500
Epoch 351/500
600/600 [==============================] - 0s 149us/step - loss: 0.0450 - acc: 0.9833 - val_loss: 0.4082 - val_acc: 0.9567
Epoch 352/500
600/600 [==============================] - 0s 153us/step - loss: 0.0448 - acc: 0.9800 - val_loss: 0.3130 - val_acc: 0.9633
Epoch 353/500
600/600 [==============================] - 0s 153us/step - loss: 0.0566 - acc: 0.9783 - val_loss: 0.3894 - val_acc: 0.9600
Epoch 354/500
600/600 [==============================] - 0s 146us/step - loss: 0.0392 - acc: 0.9850 - val_loss: 0.3879 - val_acc: 0.9600
Epoch 355/500
600/600 [==============================] - 0s 158us/step - loss: 0.0546 - acc: 0.9783 - val_loss: 0.3643 - val_acc: 0.9633
Epoch 356/500
600/600 [==============================] - 0s 152us/step - loss: 0.0336 - acc: 0.9850 - val_loss: 0.2963 - val_acc: 0.9667
Epoch 357/500
600/600 [==================

Epoch 410/500
600/600 [==============================] - 0s 145us/step - loss: 0.0366 - acc: 0.9867 - val_loss: 0.2401 - val_acc: 0.9767
Epoch 411/500
600/600 [==============================] - 0s 144us/step - loss: 0.0565 - acc: 0.9817 - val_loss: 0.1999 - val_acc: 0.9833
Epoch 412/500
600/600 [==============================] - 0s 145us/step - loss: 0.0317 - acc: 0.9883 - val_loss: 0.1902 - val_acc: 0.9833
Epoch 413/500
600/600 [==============================] - 0s 145us/step - loss: 0.0335 - acc: 0.9883 - val_loss: 0.2015 - val_acc: 0.9800
Epoch 414/500
600/600 [==============================] - 0s 145us/step - loss: 0.0280 - acc: 0.9900 - val_loss: 0.2690 - val_acc: 0.9767
Epoch 415/500
600/600 [==============================] - 0s 143us/step - loss: 0.0336 - acc: 0.9850 - val_loss: 0.2343 - val_acc: 0.9733
Epoch 416/500
600/600 [==============================] - 0s 142us/step - loss: 0.0373 - acc: 0.9883 - val_loss: 0.4247 - val_acc: 0.9567
Epoch 417/500
600/600 [==================

Epoch 470/500
600/600 [==============================] - 0s 161us/step - loss: 0.0344 - acc: 0.9850 - val_loss: 0.4884 - val_acc: 0.9533
Epoch 471/500
600/600 [==============================] - 0s 156us/step - loss: 0.0458 - acc: 0.9833 - val_loss: 0.4810 - val_acc: 0.9533
Epoch 472/500
600/600 [==============================] - 0s 151us/step - loss: 0.0221 - acc: 0.9933 - val_loss: 0.4763 - val_acc: 0.9533
Epoch 473/500
600/600 [==============================] - 0s 146us/step - loss: 0.0328 - acc: 0.9867 - val_loss: 0.4918 - val_acc: 0.9533
Epoch 474/500
600/600 [==============================] - 0s 152us/step - loss: 0.0276 - acc: 0.9917 - val_loss: 0.4566 - val_acc: 0.9633
Epoch 475/500
600/600 [==============================] - 0s 150us/step - loss: 0.0468 - acc: 0.9817 - val_loss: 0.4149 - val_acc: 0.9667
Epoch 476/500
600/600 [==============================] - 0s 148us/step - loss: 0.0297 - acc: 0.9883 - val_loss: 0.2833 - val_acc: 0.9667
Epoch 477/500
600/600 [==================

In [17]:
print (len(test_label))
print (len(test_label)*0.9867)


300
296.01
